In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
import json

import datasets
import pandas as pd
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
new_model = "Llama-2-7b-chat-spinettAI"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=8,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
)


In [ ]:
class InstructLyricsDataset:
    """Dataset for instruct fine tuning."""

    def __init__(self, input_file_path: str) -> None:
        self.input_file_path = input_file_path
        self.data = self.load_data()

    def load_data(self):
        """Load data from file."""
        with open(self.input_file_path, "r") as f:
            data = json.load(f)

        return [{"text": lyrics} for _, lyrics in data.items()]

    def _transform_text_for_finetuning(self, text: str) -> str:
        """Transform text for finetuning."""
        text_0, text_1 = text.split("\n", maxsplit=1)
        instruction = f"<s>[INST] Eres SpinettAI, un bot musical. Tienes que escribir una cancion siguiendo la siguiente frase: \n{text_0}[/INST]"
        answer = f"{text_1} </s>"
        return instruction + answer

    def get(self) -> datasets.Dataset:
        """Get dataset for instruct fine tuning."""
        pd_data = pd.DataFrame(self.data)
        dataset = datasets.Dataset.from_pandas(pd_data)
        dataset = dataset.map(
            lambda x: {"text": self._transform_text_for_finetuning(x["text"])}
        )
        return dataset

In [ ]:
dataset = InstructLyricsDataset(input_file_path="/content/lyrics.json").get()

model = AutoModelForCausalLM.from_pretrained(
    model_name, quantization_config=bnb_config, device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

trainer.train()
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model)